In [ ]:
from quadmodel.inference.forward_model import forward_model
import os
import numpy as np
import matplotlib.pyplot as plt

### Generating distributions of a summary statistic for dark matter models 

This produces curves like Figure 3 from https://ui.adsabs.harvard.edu/abs/2019MNRAS.487.5721G/abstract

In [ ]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
import matplotlib.pyplot as plt
import numpy as np


lens_model = LensModel(['EPL', 'SHEAR'])
kwargs_lens = [{'theta_E': 1.0, 'center_x': 0.0, 'center_y': 0.0, 'e1': -0.1, 'e2': 0.03, 'gamma': 2.0}, 
               {'gamma1': 0.04, 'gamma2': -0.07}]
shear_strength = np.hypot(0.04, 0.07)

source_x, source_y = 0.04, -0.02
solver = LensEquationSolver(lens_model)
x_image, y_image = solver.image_position_from_source(source_x, source_y, kwargs_lens)
magnification = lens_model.magnification(x_image, y_image, kwargs_lens)
magnification = np.absolute(magnification)
plt.scatter(x_image, y_image, color='k')
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
for (xi, yi, mi) in zip(x_image, y_image, magnification):
    plt.annotate(str(np.round(mi,2)), xy=(xi+0.1, yi+0.05))
normalized_magnifications = magnification/magnification[0]
print(normalized_magnifications)

In [ ]:
from quadmodel.data.quad_base import Quad

class SimulatedData(Quad):
    
    def __init__(self, x, y, m, sourcemodel_type='midIR_Gaussian',
                 macromodel_type='EPL_FIXED_SHEAR', kwargs_macromodel={}):
        
        delta_m = 0.001 * np.array(m) # basically no flux uncertainties
        delta_xy = [0.0001] * 4 # basically no image position uncertainties
        keep_flux_ratio_index = [0, 1, 2] # keep all three flux ratios
        zlens = 0.5
        zsource = 2.0
        
        super(SimulatedData, self).__init__(zlens, zsource, x, y, m, delta_m, delta_xy, sourcemodel_type, {}, macromodel_type,
                                    kwargs_macromodel, keep_flux_ratio_index)
        
simulated_data = SimulatedData(x_image, y_image, normalized_magnifications)



In [ ]:
output_path = os.getcwd() + '/example_summary_stat_output/'
job_index = 1
n_keep = 100
summary_statistic_tolerance = 1e4

print(simulated_data.x)
print(simulated_data.y)
print(simulated_data.m)
print(simulated_data.macromodel_type)
realization_priors = {}
realization_priors['PRESET_MODEL'] = 'CDM'
realization_priors['sigma_sub'] = ['FIXED', 0.05]
realization_priors['LOS_normalization'] = ['FIXED', 4.]
realization_priors['log_m_host'] = ['FIXED', 13.3]
realization_priors['log_mlow'] = ['FIXED', 9.0] # this sets the minimum halo mass to 10^9 for speedy demonstration purposes
realization_priors['cone_opening_angle'] = ['FIXED', 8.0]
macromodel_priors = {}
macromodel_priors['gamma_macro_prior'] = [np.random.uniform, 1.95, 2.05]
macromodel_priors['shear_strength_prior'] = [np.random.uniform, shear_strength-0.025, shear_strength+0.025]

### Run the simulation

In [ ]:
test_mode = False # make figures of the convergence and lensed images; turn off when running on a cluster
verbose = False # make print statements
forward_model(output_path, job_index, simulated_data, n_keep, realization_priors, 
              macromodel_priors, tolerance=summary_statistic_tolerance,
                  verbose=verbose, test_mode=test_mode, save_realizations=False)

### Examine the output

Plot the cumulative distribution of the summary statistic values

In [ ]:
n_files = 1
statistic = np.array([])
for i in range(1, n_files+1):
    stat = np.loadtxt(output_path+'job_'+str(i)+'/parameters.txt',skiprows=1)[:,-2]
    statistic = np.append(statistic, stat)

s = np.linspace(0, 1., 10)
x = []
for xi in s:
    n = np.sum(statistic > xi)
    x.append(n)
x = np.array(x)/len(statistic)

plt.plot(s, x)